
### `ADS500B`
### `Team 2 Final Project: Data Analysis and Preliminary Analytics`

#### Team members:
* **Muris Saab**
* **Landon Padgett**
* **Ghassan Seba**


In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

### 1. Data Importing and Pre-processing

<div class="alert alert-block alert-info">

### 1.1 
* **Import dataset** 
* **Describe characteristics such as dimensions, data types, file types, and import methods used.**
<div>

In [3]:
#Import 'online_shoppers_intention.csv' into a pandas DataFrame
onlineShop_df = pd.read_csv('online_shoppers_intention.csv')
onlineShop_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0.0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1.0,1,1,1,Returning_Visitor,False,False
1,0,0.0,0.0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2.0,2,1,2,Returning_Visitor,False,False
2,0,0.0,0.0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4.0,1,9,3,Returning_Visitor,False,False
3,0,0.0,0.0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3.0,2,2,4,Returning_Visitor,False,False
4,0,0.0,0.0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3.0,3,1,4,Returning_Visitor,True,False


In [3]:
print(onlineShop_df.dtypes, '\n') # data types
print(onlineShop_df.info(), '\n') # more info on dataframe

# print length of onlineShop_df (rows)
print('Length of Dataframe:', len(onlineShop_df),
      '\n')

# number of rows of dataframe
print('Number of Rows:', onlineShop_df.shape[0])

# number of columns of dataframe
print('Number of Columns:', onlineShop_df.shape[1])

Administrative               int64
Administrative_Duration    float64
Informational              float64
Informational_Duration     float64
ProductRelated               int64
ProductRelated_Duration    float64
BounceRates                float64
ExitRates                  float64
PageValues                 float64
SpecialDay                 float64
Month                       object
OperatingSystems           float64
Browser                      int64
Region                       int64
TrafficType                  int64
VisitorType                 object
Weekend                       bool
Revenue                       bool
dtype: object 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational           

In [4]:
# sum up all of the missing values in each row (if there are any)
print(onlineShop_df.isnull().sum())

Administrative               0
Administrative_Duration      0
Informational              128
Informational_Duration       0
ProductRelated               0
ProductRelated_Duration      0
BounceRates                  0
ExitRates                    0
PageValues                 135
SpecialDay                   0
Month                        0
OperatingSystems           123
Browser                      0
Region                       0
TrafficType                  0
VisitorType                  0
Weekend                      0
Revenue                      0
dtype: int64


### 1.2 
* **Clean** 
* **Wrangle** 
* **Handle missing data**

In [5]:
# Identify columns with missing values
cols_with_missing = onlineShop_df.columns[onlineShop_df.isna().any()].tolist()
cols_with_missing

['Informational', 'PageValues', 'OperatingSystems']

In [6]:
# Fill missing values with appropriate values
onlineShop_df['Informational'].fillna(0, inplace=True)
onlineShop_df['PageValues'].fillna(0, inplace=True)
onlineShop_df['OperatingSystems'].fillna(onlineShop_df['OperatingSystems'].mode()[0], inplace=True)

#https://vitalflux.com/pandas-impute-missing-values-mean-median-mode/

**According to the data description in the 'README.txt', missing data in the 'Informational' and 'PageValues' columns might indicate users didn't interact with any informational pages make any purchase on the website, so replacing them with zero's seems like a reasonable decision. On the other hand, the 'OperatingSystems' column appears to be a categorical (ordinal) column and we can assume that the missing data is random. In this case, a common method of dealing with missing data is mode imputation, which involves replacing the missing values with the most frequent category in the column. This method is less sensitive to outliers than using the mean.** 

In [7]:
# sum up all of the missing values in each row (to verify values were filled)
print(onlineShop_df.isnull().sum())

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64


### 1.3 Transform data appropriately using techniques such as:

* **Aggregation**
* **Normalization**
* **Feature Construction**

**Aggregation: the process of summarizing or combining multiple rows of data into a single row, usually by applying a mathematical or statistical function to the data.**

**Aggregation: Create a 'Monthly' view and 'Vistor Type'**

In [8]:
# Groupby & multiple aggregations on different columns grouped by Month
monthlySummary_df = onlineShop_df.groupby('Month').aggregate({'PageValues': ['mean', 'median'],
                                    'ProductRelated_Duration': ['mean', 'median'],
                                    'ExitRates': ['mean', 'median'],
                                    'BounceRates': ['mean', 'median'],
                                    'Revenue': 'sum'})

# Round the values to 2 decimal places
monthlySummary_df = monthlySummary_df.round(2)

monthlySummary_df

#https://sparkbyexamples.com/pandas/pandas-groupby-multiple-columns/

PageValues        ProductRelated_Duration         ExitRates         \
            mean median                    mean  median      mean median   
Month                                                                      
Aug         5.88    0.0                 1272.65  684.60      0.04   0.02   
Dec         6.74    0.0                 1111.47  611.08      0.04   0.02   
Feb         0.89    0.0                  471.01  214.92      0.07   0.05   
Jul         4.10    0.0                 1217.60  542.16      0.05   0.03   
June        3.39    0.0                 1213.38  464.31      0.06   0.04   
Mar         3.96    0.0                  812.28  438.43      0.04   0.02   
May         5.38    0.0                  981.89  535.73      0.05   0.03   
Nov         7.09    0.0                 1758.40  926.45      0.04   0.02   
Oct         8.56    0.0                 1116.98  572.87      0.03   0.02   
Sep         7.56    0.0                 1253.39  613.75      0.03   0.02   

      BounceRates        Revenue  
             mean median     sum  
Month                             
Aug          0.02   0.00      76  
Dec          0.02   0.00     216  
Feb          0.05   0.00       3  
Jul          0.02   0.01      66  
June         0.04   0.01      29  
Mar          0.02   0.00     192  
May          0.03   0.01     365  
Nov          0.02   0.00     760  
Oct          0.01   0.00     115  
Sep          0.01   0.00      86

In [9]:
# Groupby & multiple aggregations on different columns grouped by VisitorType
visitorSummary_df = onlineShop_df.groupby('VisitorType').aggregate({'PageValues': ['mean', 'median'],
                                    'ProductRelated_Duration': ['mean', 'median'],
                                    'ExitRates': ['mean', 'median'],
                                    'BounceRates': ['mean', 'median'],
                                    'Revenue': 'sum'})

# Round the values to 2 decimal places
visitorSummary_df = visitorSummary_df.round(2)

visitorSummary_df

PageValues        ProductRelated_Duration         ExitRates  \
                        mean median                    mean  median      mean   
VisitorType                                                                     
New_Visitor            10.77    0.0                  636.39  414.25      0.02   
Other                  18.19    0.0                  570.40  136.50      0.06   
Returning_Visitor       4.96    0.0                 1289.42  655.54      0.05   

                         BounceRates        Revenue  
                  median        mean median     sum  
VisitorType                                          
New_Visitor         0.01        0.01   0.00     422  
Other               0.03        0.04   0.00      16  
Returning_Visitor   0.03        0.02   0.01    1470

**Normalization of numeric data: the process of scaling the features so that they have a similar range. There are different techniques for normalization, but we will use Min-Max scaling, which scales the values between 0 and 1 for the "duration" columns**

In [10]:
# Get list of numeric columns
numerCols = onlineShop_df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Create scaler object
scaler = StandardScaler()

# Scale numeric columns
onlineShop_df[numerCols] = scaler.fit_transform(onlineShop_df[numerCols])

onlineShop_df

#https://datatofish.com/list-column-names-pandas-dataframe/
#https://www.digitalocean.com/community/tutorials/standardscaler-function-in-python
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html
#https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,-0.696993,-0.457191,-0.394026,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.315356,-0.308821,Feb,-1.237039,-0.790293,-0.894178,-0.762629,Returning_Visitor,False,False
1,-0.696993,-0.457191,-0.394026,-0.244931,-0.668518,-0.590903,-0.457683,1.171473,-0.315356,-0.308821,Feb,-0.134886,-0.207952,-0.894178,-0.514182,Returning_Visitor,False,False
2,-0.696993,-0.457191,-0.394026,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.315356,-0.308821,Feb,2.069418,-0.790293,2.437081,-0.265735,Returning_Visitor,False,False
3,-0.696993,-0.457191,-0.394026,-0.244931,-0.668518,-0.622954,0.573535,1.994610,-0.315356,-0.308821,Feb,0.967266,-0.207952,-0.477771,-0.017289,Returning_Visitor,False,False
4,-0.696993,-0.457191,-0.394026,-0.244931,-0.488636,-0.296430,-0.045196,0.142551,-0.315356,-0.308821,Feb,0.967266,0.374389,-0.894178,-0.017289,Returning_Visitor,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,0.206173,0.363075,-0.394026,-0.244931,0.478227,0.307822,-0.310366,-0.288966,0.344956,-0.308821,Dec,2.069418,2.121412,-0.894178,-0.762629,Returning_Visitor,True,False
12326,-0.696993,-0.457191,-0.394026,-0.244931,-0.601062,-0.380957,-0.457683,-0.447364,-0.315356,-0.308821,Nov,0.967266,-0.207952,-0.894178,0.976499,Returning_Visitor,True,False
12327,-0.696993,-0.457191,-0.394026,-0.244931,-0.578577,-0.528063,1.261014,0.897093,-0.315356,-0.308821,Nov,0.967266,-0.207952,-0.894178,2.218733,Returning_Visitor,True,False
12328,0.507228,-0.032916,-0.394026,-0.244931,-0.376210,-0.443536,-0.457683,-0.453140,-0.315356,-0.308821,Nov,-0.134886,-0.207952,-0.061364,1.721839,Returning_Visitor,False,False


### Although categorical data cannot be normalized because it represents discrete categories and doesn't have a continuous numerical scale. It can, however, be transformed using techniques such as one-hot encoding, which convert categorical variables into numerical formats that allow for statistical analysis to be conducted on them.

In [11]:
# # Select the categorical columns to encode
# catCols = ['OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend', 'Month']

# # Apply one-hot encoding
# onlineShop_df = pd.get_dummies(onlineShop_df, columns=catCols)

# # Display the encoded dataframe
# onlineShop_df

# #https://datagy.io/pandas-get-dummies/

### Feature construction is the process of creating new features in your data set from the existing ones.

In [12]:
# Feature construction: create a new feature 'TotalDuration' by adding up the duration of all types of pages visited
onlineShop_df['TotalDuration'] = onlineShop_df['Administrative_Duration'] + \
                                 onlineShop_df['Informational_Duration'] + onlineShop_df['ProductRelated_Duration']

onlineShop_df

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue,TotalDuration
0,-0.696993,-0.457191,-0.394026,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.315356,-0.308821,Feb,-1.237039,-0.790293,-0.894178,-0.762629,Returning_Visitor,False,False,-1.326469
1,-0.696993,-0.457191,-0.394026,-0.244931,-0.668518,-0.590903,-0.457683,1.171473,-0.315356,-0.308821,Feb,-0.134886,-0.207952,-0.894178,-0.514182,Returning_Visitor,False,False,-1.293024
2,-0.696993,-0.457191,-0.394026,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.315356,-0.308821,Feb,2.069418,-0.790293,2.437081,-0.265735,Returning_Visitor,False,False,-1.326469
3,-0.696993,-0.457191,-0.394026,-0.244931,-0.668518,-0.622954,0.573535,1.994610,-0.315356,-0.308821,Feb,0.967266,-0.207952,-0.477771,-0.017289,Returning_Visitor,False,False,-1.325076
4,-0.696993,-0.457191,-0.394026,-0.244931,-0.488636,-0.296430,-0.045196,0.142551,-0.315356,-0.308821,Feb,0.967266,0.374389,-0.894178,-0.017289,Returning_Visitor,True,False,-0.998552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,0.206173,0.363075,-0.394026,-0.244931,0.478227,0.307822,-0.310366,-0.288966,0.344956,-0.308821,Dec,2.069418,2.121412,-0.894178,-0.762629,Returning_Visitor,True,False,0.425966
12326,-0.696993,-0.457191,-0.394026,-0.244931,-0.601062,-0.380957,-0.457683,-0.447364,-0.315356,-0.308821,Nov,0.967266,-0.207952,-0.894178,0.976499,Returning_Visitor,True,False,-1.083079
12327,-0.696993,-0.457191,-0.394026,-0.244931,-0.578577,-0.528063,1.261014,0.897093,-0.315356,-0.308821,Nov,0.967266,-0.207952,-0.894178,2.218733,Returning_Visitor,True,False,-1.230185
12328,0.507228,-0.032916,-0.394026,-0.244931,-0.376210,-0.443536,-0.457683,-0.453140,-0.315356,-0.308821,Nov,-0.134886,-0.207952,-0.061364,1.721839,Returning_Visitor,False,False,-0.721382


### Reduce redundant data: drop the individual duration features since we have created a new feature 'TotalDuration'

In [13]:
# Drop unwanted columns
onlineShop_df = onlineShop_df.drop(['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration'], axis=1)
onlineShop_df

,Administrative,Informational,ProductRelated,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue,TotalDuration
0,-0.696993,-0.394026,-0.691003,3.667189,3.229316,-0.315356,-0.308821,Feb,-1.237039,-0.790293,-0.894178,-0.762629,Returning_Visitor,False,False,-1.326469
1,-0.696993,-0.394026,-0.668518,-0.457683,1.171473,-0.315356,-0.308821,Feb,-0.134886,-0.207952,-0.894178,-0.514182,Returning_Visitor,False,False,-1.293024
2,-0.696993,-0.394026,-0.691003,3.667189,3.229316,-0.315356,-0.308821,Feb,2.069418,-0.790293,2.437081,-0.265735,Returning_Visitor,False,False,-1.326469
3,-0.696993,-0.394026,-0.668518,0.573535,1.994610,-0.315356,-0.308821,Feb,0.967266,-0.207952,-0.477771,-0.017289,Returning_Visitor,False,False,-1.325076
4,-0.696993,-0.394026,-0.488636,-0.045196,0.142551,-0.315356,-0.308821,Feb,0.967266,0.374389,-0.894178,-0.017289,Returning_Visitor,True,False,-0.998552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,0.206173,-0.394026,0.478227,-0.310366,-0.288966,0.344956,-0.308821,Dec,2.069418,2.121412,-0.894178,-0.762629,Returning_Visitor,True,False,0.425966
12326,-0.696993,-0.394026,-0.601062,-0.457683,-0.447364,-0.315356,-0.308821,Nov,0.967266,-0.207952,-0.894178,0.976499,Returning_Visitor,True,False,-1.083079
12327,-0.696993,-0.394026,-0.578577,1.261014,0.897093,-0.315356,-0.308821,Nov,0.967266,-0.207952,-0.894178,2.218733,Returning_Visitor,True,False,-1.230185
12328,0.507228,-0.394026,-0.376210,-0.457683,-0.453140,-0.315356,-0.308821,Nov,-0.134886,-0.207952,-0.061364,1.721839,Returning_Visitor,False,False,-0.721382


### Perform need-based discretization on continuous values and bin them into segments labeled 'low', 'medium', 'high', 'very_high. The new discretized values provide information on the relative position of each data point within the range of values for each continuous feature. Each data point is assigned to one of four categories - 1 = 'low', 2 = 'medium', 3 = 'high', or 4 = 'very_high', depending on its value in the original continuous feature column.

### The categories represent the quartiles of the data, i.e., the lowest 25% of values are in the 'low' category, the next 25% of values are in the 'medium' category, the next 25% of values are in the 'high' category, and the top 25% of values are in the 'very_high' category.

### The new discretized values provide a simplified representation of the original continuous feature values, which can be useful for performing analyses that require categorical data. For example, we can use the discretized values to perform frequency analysis or cross-tabulation to understand the relationship between the different variables.

In [14]:
# Define the columns to discretize
cols_to_discretize = ["Administrative", "Informational", "ProductRelated", "BounceRates", "ExitRates", 
                      "PageValues", "SpecialDay", "TotalDuration"]

# # Convert the columns to a numeric data type
# onlineShop_df[cols_to_discretize] = onlineShop_df[cols_to_discretize].apply(pd.to_numeric, errors='coerce')

# Perform need-based discretization for each column
for col in cols_to_discretize:
    onlineShop_df[col+'_discretized'] = pd.cut(onlineShop_df[col], bins=4, labels=[1,2,3,4])

onlineShop_df

#https://pbpython.com/pandas-qcut-cut.html
#https://pandas.pydata.org/docs/reference/api/pandas.cut.html

,Administrative,Informational,ProductRelated,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,...,Revenue,TotalDuration,Administrative_discretized,Informational_discretized,ProductRelated_discretized,BounceRates_discretized,ExitRates_discretized,PageValues_discretized,SpecialDay_discretized,TotalDuration_discretized
0,-0.696993,-0.394026,-0.691003,3.667189,3.229316,-0.315356,-0.308821,Feb,-1.237039,-0.790293,...,False,-1.326469,1,1,1,4,4,1,1,1
1,-0.696993,-0.394026,-0.668518,-0.457683,1.171473,-0.315356,-0.308821,Feb,-0.134886,-0.207952,...,False,-1.293024,1,1,1,1,3,1,1,1
2,-0.696993,-0.394026,-0.691003,3.667189,3.229316,-0.315356,-0.308821,Feb,2.069418,-0.790293,...,False,-1.326469,1,1,1,4,4,1,1,1
3,-0.696993,-0.394026,-0.668518,0.573535,1.994610,-0.315356,-0.308821,Feb,0.967266,-0.207952,...,False,-1.325076,1,1,1,1,3,1,1,1
4,-0.696993,-0.394026,-0.488636,-0.045196,0.142551,-0.315356,-0.308821,Feb,0.967266,0.374389,...,False,-0.998552,1,1,1,1,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,0.206173,-0.394026,0.478227,-0.310366,-0.288966,0.344956,-0.308821,Dec,2.069418,2.121412,...,False,0.425966,1,1,1,1,1,1,1,1
12326,-0.696993,-0.394026,-0.601062,-0.457683,-0.447364,-0.315356,-0.308821,Nov,0.967266,-0.207952,...,False,-1.083079,1,1,1,1,1,1,1,1
12327,-0.696993,-0.394026,-0.578577,1.261014,0.897093,-0.315356,-0.308821,Nov,0.967266,-0.207952,...,False,-1.230185,1,1,1,2,2,1,1,1
12328,0.507228,-0.394026,-0.376210,-0.457683,-0.453140,-0.315356,-0.308821,Nov,-0.134886,-0.207952,...,False,-0.721382,1,1,1,1,1,1,1,1
